Julia Kang - jkang613@gmail.com - AMLI 2019 - Mills College

In [1]:
import pandas as pd
import numpy as np
import multiprocessing as mp
import random
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model
# from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from math import sqrt #for RMSE


## Model: SGDClassifier Model

**Logistic vs SGDClassifier:** Logistic Regression by default uses Gradient Descent and as such it would be better to use SGD Classifier on larger data sets. One another reason you might want to use SGD Classifier is, logistic regression, in its vanilla sklearn form, won’t work if you can’t hold the dataset in RAM but SGD will still work.

For sufficiently large datasets, it is best to implement SGD Classifier instead of Logistic Classifier to produce similar results in much less time.

In [2]:
# raw_df = pd.read_csv('numeric_rand_sample.csv')
raw_df = pd.read_csv('choice_features.csv')
raw_df = raw_df.drop('Unnamed: 0', axis = 1) 

raw_df.head()
# len(sub_df)

,click,C1,banner_pos,site_domain,site_category,app_domain,app_category,device_id,device_ip,device_model,...,C15,C16,date,time,user_id,device_ip_count,device_id_count,user_count,user_hour_count,user_bagged
0,0,1005,0,227,3,12,0,438,1599,138,...,320,50,0,0,1681,2,2,2,95,232
1,0,1005,1,7,12,12,0,438,611,393,...,320,50,0,0,638,30,30,30,95,232
2,0,1005,0,225,4,15,2,438,1212,505,...,320,50,0,0,1264,1499,1499,1499,95,255
3,0,1005,0,125,3,12,0,438,1034,373,...,300,250,0,0,1081,97,97,97,95,232
4,0,1005,0,227,3,12,0,438,3474,746,...,320,50,0,0,3663,2,2,2,95,232


In [3]:
FEATURES = []

for col in raw_df.columns:
    if np.issubdtype(raw_df[col].dtypes , np.number):
        FEATURES.append(col)

FEATURES.remove('click')
TARGET = 'click'

In [4]:
# split the data into test and train
train, test  = train_test_split(
  raw_df,                        # split our digits dataframe
  stratify=raw_df[TARGET],       # stratify by the 'click' column   
  test_size=0.2,                # 20% of the data should be held for testing
  random_state=42               # hard-coded random state for repeatability in the example
)

train.head()

,click,C1,banner_pos,site_domain,site_category,app_domain,app_category,device_id,device_ip,device_model,...,C15,C16,date,time,user_id,device_ip_count,device_id_count,user_count,user_hour_count,user_bagged
4114,0,1005,0,225,4,3,12,438,931,406,...,320,50,9,18,963,1499,1499,1499,170,148
1250,0,1002,0,274,4,12,0,266,2147,564,...,320,50,2,14,1724,1,1,1,211,232
3862,0,1005,1,83,12,12,0,438,1613,123,...,320,50,9,7,1694,24,24,24,174,232
2094,1,1005,0,275,1,12,0,438,451,864,...,320,50,5,2,479,720,720,720,136,232
670,1,1005,0,225,4,7,2,543,922,67,...,320,50,1,10,3449,1499,1499,1499,232,248


In [5]:
train_features = np.array(train[FEATURES])
train_labels = np.array(train[TARGET])
test_features = np.array(test[FEATURES])
test_labels = np.array(test[TARGET])

In [6]:
sgd_classifier = linear_model.SGDClassifier(
    loss='log',
    random_state=2, # Specifying random state allow us to get repeatable outcomes
    tol=1e-3, 
    max_iter=500,
    alpha=0.001
) 
 
sgd_classifier.fit(train_features, train_labels.ravel())

# sgd_classifier.partial_fit(train_features, train_labels.ravel(), classes = [0,1]) #partial_fit SUCKED

SGDClassifier(alpha=0.001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=500,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=2, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [7]:
# get the predictions using our model
sgd_predictions = sgd_classifier.predict(test_features)

# we can use the scores to set thresholds on the predictions made
sgd_scores = sgd_classifier.decision_function(test_features)
RMSE = sqrt(metrics.mean_squared_error(test_labels, sgd_predictions))
print(("Model using SGDClassifier has an RMSE of {} ").format(RMSE))
# 0.4158645213482616 - base
# 0.41513549509867104 - loss = log

LL = sqrt(metrics.log_loss(test_labels, sgd_predictions))
print(("Model using SGDClassifier has an LL of {} ").format(LL))
# Model using SGDClassifier has an LL of 2.425344088859301 - Using Ryan's Data

Model using SGDClassifier has an RMSE of 0.41333795811957696 
Model using SGDClassifier has an LL of 2.429174976856048 


In [8]:
# # loop through different C (alpha) values to see which gives us the largest AUC value
# alphas = [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1]

# LLS = []
# AUCS = []

# for a in alphas:
#     sgd_classifier = linear_model.SGDClassifier(
#         alpha = a,
#         loss='log',
#         random_state=2, # Specifying random state allow us to get repeatable outcomes
#         tol=1e-3, 
#         max_iter=500
#     ) 

#     sgd_classifier.fit(train_features, train_labels.ravel())
#     sgd_predictions = sgd_classifier.predict(test_features)
#     sgd_ll = sqrt(metrics.log_loss(test_labels, sgd_predictions))
#     LLS.append(sgd_ll)
#     sgd_fpr, sgd_tpr, sgd_thresholds = metrics.roc_curve(test_labels, sgd_predictions)
#     sgd_roc_auc = metrics.auc(sgd_fpr, sgd_tpr)
#     AUCS.append(sgd_roc_auc)
    
#     print(("Model where alpha is {} is completed").format(a))

# print(LLS)

# # plot to visiual the different AUC in relation to alpha 
# plt.plot(AUCS)
# plt.xticks(range(len(alphas)), alphas)
# plt.ylabel('AUC')
# plt.xlabel('alpha')
# plt.show()